In [1]:
import os

import numpy as np
import tensorflow as tf

tf.random.set_seed(0)
tf.random.set_global_generator(tf.random.Generator.from_seed(0))

2024-12-03 15:48:22.806373: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 15:48:22.807891: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 15:48:22.836722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 15:48:22.837483: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 15:48:24.741084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
def save_datasets(num_chunks, chunk_size=3):

    for i in range(num_chunks):
        start = i * chunk_size
        end = (i + 1) * chunk_size

        def generator():
            for i in range(start, end):
                yield i

        dataset = tf.data.Dataset.from_generator(generator, output_types=tf.int64)

        tempdir = './temp_datadir/'
        filename = f'{tempdir}/ds_{start:03d}-{end:03d}'
        os.makedirs(tempdir, exist_ok=True)

        dataset.save(filename)

save_datasets(10)

In [3]:
def add_noise(x):
    noise = tf.random.normal(tf.shape(x), stddev=0.01)
    return x + noise

In [35]:
def get_datasets():
    tempdir = './temp_datadir/'
    filenames = [
        os.path.join(tempdir, f)
        for f in os.listdir(tempdir)
    ]
    element_spec = tf.data.Dataset.load(filenames[0]).element_spec

    list_ds = tf.data.Dataset.from_tensor_slices(filenames)
    list_ds = list_ds.shuffle(10, seed=0)
    dataset = list_ds.interleave(
        lambda filename: tf.data.Dataset.load(filename, element_spec=element_spec),
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=True,
    )
    dataset = dataset.repeat(2)
    dataset = dataset.shuffle(10, reshuffle_each_iteration=True, seed=0)

    # Cast to float32
    dataset = dataset.map(
        lambda x: tf.cast(x, tf.float32),
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=True,
    )

    dataset = dataset.map(
        add_noise,
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=True,
    )

    dataset = dataset.batch(5)
    dataset = dataset.as_numpy_iterator()
    return dataset

In [46]:
for i in get_datasets():
    for j in i:
        print(f"{j:.2f}", end=' -- ')
    print(f"{i.std():.5e}")

21.00 -- 4.01 -- -0.01 -- 13.99 -- 28.99 -- 1.06683e+01
3.02 -- 27.99 -- 1.98 -- 5.00 -- 13.00 -- 9.70068e+00
10.01 -- 21.99 -- 15.00 -- 27.00 -- 9.00 -- 6.94333e+00
0.98 -- 25.00 -- 16.00 -- 23.01 -- 8.00 -- 9.05343e+00
19.99 -- 6.03 -- -0.00 -- 6.99 -- 11.01 -- 6.61287e+00
15.00 -- 18.01 -- 1.01 -- 9.01 -- 26.00 -- 8.41930e+00
11.99 -- 19.00 -- 10.00 -- 16.00 -- 6.00 -- 4.54318e+00
8.02 -- 1.99 -- 24.03 -- 14.00 -- 17.99 -- 7.65744e+00
16.99 -- 13.00 -- 20.00 -- 11.00 -- 24.00 -- 4.68901e+00
3.01 -- 7.00 -- 27.01 -- 23.01 -- 21.00 -- 9.43168e+00
25.00 -- 26.00 -- 11.99 -- 17.01 -- 28.99 -- 6.30438e+00
28.01 -- 21.99 -- 3.98 -- 18.98 -- 5.00 -- 9.52753e+00


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 11632008680831010402
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 6535569743691840332
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing t